In [ ]:
import glob
import os
import shutil

from astropy.io import fits
from astropy import stats
from astropy.time import Time
import numpy as np
import pandas as pd

"The data in Figure 10 are from program 13078 visits A3, A6, A7, and A8 and taken on November 15
and 16, 2012. Images were combined in pairs with STSDAS CRREJ, and the mean pixel value was
computed for the 1024x1024 pixel area nearest the C amplifier using IMSTAT with 3 iterations of 3
sigma rejection. "

In [ ]:
postflash_data = pd.read_pickle('/grp/hst/wfc3u/postflash_2021/flt_postflash_with_stats.pkl')
low_sub_pf_isr_13078_nov15 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['propid'] == 13078) & (postflash_data['obsdate'] == '2012-11-15')]
low_sub_pf_isr_13078_nov16 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['propid'] == 13078) & (postflash_data['obsdate'] == '2012-11-16')]

both_dates = [low_sub_pf_isr_13078_nov15, low_sub_pf_isr_13078_nov16]

low_sub_pf_isr_13078 = pd.concat(both_dates)
#print(low_sub_pf_isr_13078)

In [ ]:
paths = low_sub_pf_isr_13078.path.tolist() 

means = []
sig_means = []
for f in paths:
    path = f
    data = fits.getdata(path)
    data_clip_1 = stats.sigma_clip(data, 3)
    data_clip_2 = stats.sigma_clip(data_clip_1, 3)
    data_clip_3 = stats.sigma_clip(data_clip_2, 3)
    mean = np.mean(data)
    sig_mean = np.mean(data_clip_3)
    means.append(mean)
    sig_means.append(sig_mean)

dict = {'path': paths, 'mean': means, 'sig_mean': sig_means}
path_and_mean_sub = pd.DataFrame(dict)

In [ ]:
low_sub_pf_isr_13078['count_rate'] = (low_sub_pf_isr_13078['sig_mean']/low_sub_pf_isr_13078['flash_dur']) * low_sub_pf_isr_13078['flash_dur']

In [ ]:
low_sub_pf_isr_13078